In [ ]:
!pip install torch torchvision torchaudio
!pip install geoopt
!pip install networkx


## WORDNET and FB15k

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from torch import optim
import geoopt
from geoopt import ManifoldParameter
import os
from tqdm import tqdm
import pandas as pd



#change here
dataset = 'fb15k' # replace with wordnet



##############################################
# Data Loading and Preprocessing
##############################################
def load_fb15k_data(train_path, valid_path, test_path):
    def read_triples(path, ent2id, rel2id):
        triples = []
        with open(path, 'r') as f:
            for line in f:
                h, r, t = line.strip().split('\t')
                if h not in ent2id:
                    ent2id[h] = len(ent2id)
                if t not in ent2id:
                    ent2id[t] = len(ent2id)
                if r not in rel2id:
                    rel2id[r] = len(rel2id)
                triples.append((ent2id[h], rel2id[r], ent2id[t]))
        return triples

    ent2id = {}
    rel2id = {}
    train_triples = read_triples(train_path, ent2id, rel2id)
    valid_triples = read_triples(valid_path, ent2id, rel2id)
    test_triples = read_triples(test_path, ent2id, rel2id)

    G = nx.Graph()
    G.add_nodes_from(range(len(ent2id)))
    for h, r, t in train_triples:
        G.add_edge(h, t)

    return ent2id, rel2id, G, train_triples, valid_triples, test_triples

def load_wordnet_data(train_path, valid_path, test_path):
    def read_triples(path, ent2id, rel2id):
        triples = []
        df = pd.read_csv(path, header=None, names=["head", "relation", "tail"])
        for _, row in df.iterrows():
            h, r, t = row["head"], row["relation"], row["tail"]
            if h not in ent2id:
                ent2id[h] = len(ent2id)
            if t not in ent2id:
                ent2id[t] = len(ent2id)
            if r not in rel2id:
                rel2id[r] = len(rel2id)
            triples.append((ent2id[h], rel2id[r], ent2id[t]))
        return triples

    ent2id = {}
    rel2id = {}
    train_triples = read_triples(train_path, ent2id, rel2id)
    valid_triples = read_triples(valid_path, ent2id, rel2id)
    test_triples = read_triples(test_path, ent2id, rel2id)

    G = nx.Graph()
    G.add_nodes_from(range(len(ent2id)))
    for h, r, t in train_triples:
        G.add_edge(h, t)

    return ent2id, rel2id, G, train_triples, valid_triples, test_triples





# Load data
if dataset == 'fb15k':
    train_path = 'FB15k_train.txt'
    valid_path = 'FB15k_valid.txt'
    test_path = 'FB15k_test.txt'
    ent2id, rel2id, G, train_triples, valid_triples, test_triples = load_fb15k_data(train_path, valid_path, test_path) # fb15k

elif dataset == 'wordnet':
    train_path = 'wordnet_train.csv'
    valid_path = 'wordnet_valid.csv'
    test_path = 'wordnet_test.csv'
    ent2id, rel2id, G, train_triples, valid_triples, test_triples = load_wordnet_data(train_path, valid_path, test_path) # wordnet


num_entities = len(ent2id)
num_relations = len(rel2id)

print(f"Number of Entities: {num_entities}")
print(f"Number of Relations: {num_relations}")
print(f"Number of Training Triples: {len(train_triples)}")
print(f"Number of Validation Triples: {len(valid_triples)}")
print(f"Number of Test Triples: {len(test_triples)}")

from scipy.sparse import csr_matrix, diags
import numpy as np

# Convert adjacency matrix to a sparse matrix
adj = nx.to_scipy_sparse_array(G, nodelist=range(num_entities)).tocsc()

# Add self-loops
adj += diags([1e-5] * num_entities)

# Compute degree
degrees = np.array(adj.sum(axis=1)).flatten()

# Compute D^-0.5 (inverse square root of degree matrix)
inv_sqrt_deg = np.power(degrees, -0.5)
inv_sqrt_deg[np.isinf(inv_sqrt_deg)] = 0
D_inv_sqrt = diags(inv_sqrt_deg)

# Normalize adjacency matrix: D^-0.5 * A * D^-0.5
norm_adj = D_inv_sqrt @ adj @ D_inv_sqrt

# Convert back to a PyTorch tensor
# To save memory, keep it on CPU if possible or use sparse tensors
norm_adj = torch.tensor(norm_adj.toarray(), dtype=torch.float32)  # If GPU is available, use .to(device)

# Node Features: Use lower-dimensional learnable embeddings instead of one-hot
embedding_dim = 128  # Reduced dimensionality
# Initialize features as learnable parameters
features = nn.Parameter(torch.randn(num_entities, embedding_dim), requires_grad=False)  # Initialize on CPU

# Move tensors to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
features, norm_adj = features.to(device), norm_adj.to(device)

print(f"Using device: {device}")


In [ ]:
##############################################
# Hyperbolic GCN and Link Prediction Model
##############################################

class HyperbolicLinear(nn.Module):
    """
    Hyperbolic linear layer using the Poincaré Ball model.
    """
    def __init__(self, manifold, in_features, out_features, bias=True):
        super(HyperbolicLinear, self).__init__()
        self.manifold = manifold
        self.in_features = in_features
        self.out_features = out_features
        # ManifoldParameter for weight in Euclidean space
        self.weight = ManifoldParameter(torch.randn(out_features, in_features) * 0.01, manifold=geoopt.Euclidean())
        if bias:
            self.bias = nn.Parameter(torch.zeros(out_features))
        else:
            self.register_parameter('bias', None)

    def forward(self, x):
        # x is on manifold. Map to tangent space at 0, apply linear, map back
        x_tan = self.manifold.logmap0(x, dim=-1)
        out = F.linear(x_tan, self.weight, self.bias)
        out = self.manifold.expmap0(out, dim=-1)
        return out

class HyperbolicGCNLayer(nn.Module):
    def __init__(self, manifold, in_features, out_features):
        super(HyperbolicGCNLayer, self).__init__()
        self.manifold = manifold
        self.lin = HyperbolicLinear(manifold, in_features, out_features)

    def forward(self, x, adj):
        # x on manifold
        x_tan = self.manifold.logmap0(x, dim=-1)
        x_agg_tan = adj @ x_tan  # Aggregate in tangent space
        x_agg = self.manifold.expmap0(x_agg_tan, dim=-1)
        return self.lin(x_agg)




############################################################################################


class HyperbolicGCN(nn.Module):
    def __init__(self, num_features, hidden_dim, c=1.0):
        super(HyperbolicGCN, self).__init__()
        # Initialize the manifold inside the model to ensure its parameters are included
        self.manifold = geoopt.PoincareBall(c=c, learnable=True)
        self.layer1 = HyperbolicGCNLayer(self.manifold, num_features, hidden_dim)
        self.layer2 = HyperbolicGCNLayer(self.manifold, hidden_dim, hidden_dim)

    def forward(self, x, adj):
        x = self.manifold.expmap0(x, dim=-1)  # Ensure x is on manifold
        x = self.layer1(x, adj)
        # Hyperbolic activation: apply tanh in tangent space
        x_tan = self.manifold.logmap0(x, dim=-1)
        x_tan = torch.tanh(x_tan)
        x = self.manifold.expmap0(x_tan, dim=-1)
        x = self.layer2(x, adj)
        return x

from geoopt.manifolds import PoincareBall

class LinkPredictionModel(nn.Module):
    def __init__(self, num_features, hidden_dim, num_relations, c=1.0):
        super(LinkPredictionModel, self).__init__()
        self.manifold = PoincareBall(c=c)
        self.node_emb = geoopt.ManifoldParameter(
            self.manifold.random((num_features, hidden_dim), std=1e-2),
            manifold=self.manifold
        )
        self.gcn = HyperbolicGCN(num_features, hidden_dim, c=c)
        self.classifier = nn.Linear(hidden_dim * 2, num_relations)
        nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, x, adj):
        node_emb = self.gcn(x, adj)  # [num_nodes, hidden_dim]
        return node_emb

    def classify(self, head_idx, tail_idx, node_emb):
        # Retrieve embeddings
        head_emb = node_emb[head_idx]     # [batch_size, hidden_dim]
        tail_emb = node_emb[tail_idx]     # [batch_size, hidden_dim]
        # Concatenate head and tail embeddings
        combined = torch.cat([head_emb, tail_emb], dim=1)  # [batch_size, hidden_dim * 2]
        scores = self.classifier(combined)  # [batch_size, num_relations]
        return scores

##############################################
# Training and Evaluation Utilities
##############################################

def prepare_data(triples):
    """
    Prepare tensors for head, tail, and relation indices.
    """
    triples = torch.tensor(triples, dtype=torch.long, device=device)
    heads = triples[:, 0]
    relations = triples[:, 1]
    tails = triples[:, 2]
    return heads, tails, relations

def train_epoch(model, optimizer, criterion, heads, tails, relations, node_emb):
    model.train()
    optimizer.zero_grad()

    # Get relation scores
    scores = model.classify(heads, tails, node_emb)  # [batch_size, num_relations]

    # Compute loss
    loss = criterion(scores, relations)
    loss.backward()
    optimizer.step()

    return loss.item()

def evaluate(model, criterion, heads, tails, relations, node_emb):
    model.eval()
    with torch.no_grad():
        scores = model.classify(heads, tails, node_emb)  # [batch_size, num_relations]
        loss = criterion(scores, relations).item()
        preds = torch.argmax(scores, dim=1)
        correct = (preds == relations).sum().item()
        total = relations.size(0)
        accuracy = correct / total
    return loss, accuracy

In [ ]:
# OPTIMIZER CODING

# Riemannian Adagrad
import torch
from torch.optim import Optimizer

class RiemannianAdagrad(Optimizer):
    def __init__(self, params, lr=1e-2, eps=1e-10, weight_decay=0):
        defaults = dict(lr=lr, eps=eps, weight_decay=weight_decay)
        super().__init__(params, defaults)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue

                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('RiemannianAdagrad does not support sparse gradients')

                # Retrieve manifold
                manifold = getattr(p, "manifold", None)
                if manifold is None:
                    raise RuntimeError("Parameter does not have a manifold defined")

                # Initialize state
                state = self.state[p]
                if "sum_sq" not in state:
                    state["sum_sq"] = torch.zeros_like(p.data)

                sum_sq = state["sum_sq"]

                # Update sum of squared gradients
                sum_sq.addcmul_(grad, grad, value=1)

                # Apply weight decay if necessary
                if group["weight_decay"] != 0:
                    grad = grad.add(p.data, alpha=group["weight_decay"])

                # Compute update
                update = grad / (sum_sq.sqrt().add(group["eps"]))

                # Riemannian update
                update = -group["lr"] * update
                p.data = manifold.proj(manifold.expmap(update, p.data))

        return loss

# Mixed-precision Riemannian optimizer
from torch.cuda.amp import GradScaler

scaler = GradScaler()

class MixedPrecisionRiemannianAdam(geoopt.optim.RiemannianAdam):
    def step(self, closure=None):
        """
        Override step to use gradient scaling for mixed precision.
        """
        if closure is not None:
            loss = closure()
        else:
            loss = None

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is not None:
                    # Scale the gradient back to normal scale
                    p.grad.data = scaler.scale(p.grad.data)

        super().step(closure)
        return loss

In [ ]:
################ criterion = nn.CrossEntropyLoss(), Multiclass #################################
# SET OPTIMIZER, LEARNING RATE, and C INITIAL VALUE

##############################################
# Training Loop
##############################################

# Initialize the model
hidden_dim = 128  # Reduced hidden dimension
model = LinkPredictionModel(num_features=embedding_dim, hidden_dim=hidden_dim, num_relations=num_relations, c=1.0).to(device)

# Initialize input: ensure features are on the manifold
with torch.no_grad():
    x = model.gcn.manifold.expmap0(features, dim=-1)

# Optimizer Choices:
riemannian_adam = geoopt.optim.RiemannianAdam(model.parameters(), lr=1e-2)
riemannian_sgd = geoopt.optim.RiemannianSGD(model.parameters(), lr=1e-1)
riemannian_mixed_precision = MixedPrecisionRiemannianAdam(model.parameters(), lr=1e-1)
riemannian_adagrad = RiemannianAdagrad(model.parameters(), lr=1e-2) # error currently

# SET OPTIMIZER:
optimizer = riemannian_mixed_precision

# Define loss function for multi-class link prediction
criterion = nn.CrossEntropyLoss()

# Prepare training, validation, and test data
train_heads, train_tails, train_relations = prepare_data(train_triples)
valid_heads, valid_tails, valid_relations = prepare_data(valid_triples)
test_heads, test_tails, test_relations = prepare_data(test_triples)

num_epochs = 1500
best_val_accuracy = 0.0
best_test_accuracy = 0.0

curvatures, train_losses, val_losses, val_accuracies, test_accuracies = [], [], [], [], []

print("\nStarting Training...\n")
for epoch in tqdm(range(1, num_epochs + 1)):
    # Forward pass to get node embeddings
    node_emb = model(features, norm_adj)  # [num_nodes, hidden_dim]

    # Training
    train_loss = train_epoch(model, optimizer, criterion, train_heads, train_tails, train_relations, node_emb)

    # Evaluation on Validation Set
    val_loss, val_accuracy = evaluate(model, criterion, valid_heads, valid_tails, valid_relations, node_emb)

    # Early Stopping and Best Model Tracking
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        # Evaluate on Test Set
        test_loss, test_accuracy = evaluate(model, criterion, test_heads, test_tails, test_relations, node_emb)
        best_test_accuracy = test_accuracy


    # Print Epoch Results
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_accuracy:.4f}, Best Test Acc={best_test_accuracy:.4f}, Curvature: {model.gcn.manifold.c.item():.6f}")

    curvatures.append(model.gcn.manifold.c.item())
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    test_accuracies.append(test_accuracy)

print("\nTraining Completed.")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")
print(f"Corresponding Test Accuracy: {best_test_accuracy:.4f}")
print(f"Curvature: {model.gcn.manifold.c.item():.6f}")

##############################################
# Inference and Evaluation
##############################################

def predict_relation(model, head_idx, tail_idx, node_emb):
    model.eval()
    with torch.no_grad():
        scores = model.classify(head_idx, tail_idx, node_emb)  # [batch_size, num_relations]
        predicted_relations = torch.argmax(scores, dim=1)
    return predicted_relations

def evaluate_full(model, criterion, heads, tails, relations, node_emb):
    model.eval()
    with torch.no_grad():
        scores = model.classify(heads, tails, node_emb)  # [batch_size, num_relations]
        loss = criterion(scores, relations).item()
        preds = torch.argmax(scores, dim=1)
        correct = (preds == relations).sum().item()
        total = relations.size(0)
        accuracy = correct / total
    return loss, accuracy

# Example Inference
sample_head = train_triples[0][0]
sample_tail = train_triples[0][2]
sample_relation = train_triples[0][1]

predicted_relation = predict_relation(model, torch.tensor([sample_head], device=device),
                                      torch.tensor([sample_tail], device=device),
                                      node_emb)

print("\nSample Inference:")
print(f"Head Entity ID: {sample_head}, Tail Entity ID: {sample_tail}")
print(f"Actual Relation ID: {sample_relation}, Predicted Relation ID: {predicted_relation.item()}")

In [ ]:
##########################################################
# PLOTTING
##########################################################
import matplotlib.pyplot as plt
epochs = list(range(1, num_epochs + 1))
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot training loss and test accuracy on the primary y-axis
ax1.plot(epochs, train_losses, label="Train Loss", color="blue", linestyle="-")
ax1.plot(epochs, test_accuracies, label="Best Test Accuracy", color="red", linestyle="--")
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Train Loss / Best Test Accuracy", color="black")
ax1.tick_params(axis="y", labelcolor="black")
ax1.legend(loc="upper left")


# Add a horizontal line at y=1
ax1.axhline(y=1, color="lightcoral", linestyle="--", linewidth=1, label="y=1")

# Add a horizontal line at y=0
ax1.axhline(y=0, color="grey", linestyle="--", linewidth=1, label="y=1")

# Plot curvature on a secondary y-axis
ax2 = ax1.twinx()
ax2.plot(epochs, curvatures, label="Curvature", color="green", linestyle=":")
ax2.set_ylabel("Curvature", color="green")
ax2.tick_params(axis="y", labelcolor="green")
ax2.legend(loc="upper right")

# Add title and grid
plt.title("Training Progress: Train Loss, Best Test Accuracy, and Curvature")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.ylim(0, 1.5)
plt.tight_layout()
plt.show()


##############################################
# Inference and Evaluation
##############################################

def predict_relation(model, head_idx, tail_idx, node_emb):
    model.eval()
    with torch.no_grad():
        scores = model.classify(head_idx, tail_idx, node_emb)  # [batch_size, num_relations]
        predicted_relations = torch.argmax(scores, dim=1)
    return predicted_relations

def evaluate_full(model, criterion, heads, tails, relations, node_emb):
    model.eval()
    with torch.no_grad():
        scores = model.classify(heads, tails, node_emb)  # [batch_size, num_relations]
        loss = criterion(scores, relations).item()
        preds = torch.argmax(scores, dim=1)
        correct = (preds == relations).sum().item()
        total = relations.size(0)
        accuracy = correct / total
    return loss, accuracy

# Example Inference
sample_head = train_triples[0][0]
sample_tail = train_triples[0][2]
sample_relation = train_triples[0][1]

predicted_relation = predict_relation(model, torch.tensor([sample_head], device=device),
                                      torch.tensor([sample_tail], device=device),
                                      node_emb)

print("\nSample Inference:")
print(f"Head Entity ID: {sample_head}, Tail Entity ID: {sample_tail}")
print(f"Actual Relation ID: {sample_relation}, Predicted Relation ID: {predicted_relation.item()}")

In [ ]:
# EMBEDDING VISUALS
# 12/13: 6:50 PM
# valentina

import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from tqdm import tqdm
import geoopt

# Assuming LinkPredictionModel and necessary imports are already defined
# Let's initialize the model and other training components

hidden_dim = 128  # Reduced hidden dimension
num_epochs = 500
best_val_accuracy = 0.0
best_test_accuracy = 0.0

# Initialize the model
model = LinkPredictionModel(num_features=embedding_dim, hidden_dim=hidden_dim, num_relations=num_relations, c=1.0).to(device)

# Optimizer setup (Riemannian mixed precision)
optimizer = geoopt.optim.RiemannianAdam(model.parameters(), lr=1e-2)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Prepare data
train_heads, train_tails, train_relations = prepare_data(train_triples)
valid_heads, valid_tails, valid_relations = prepare_data(valid_triples)
test_heads, test_tails, test_relations = prepare_data(test_triples)

curvatures, train_losses, val_losses, val_accuracies, test_accuracies = [], [], [], [], []

# Initialize PCA for visualization
pca = PCA(n_components=2)

# Training Loop
print("\nStarting Training...\n")
for epoch in tqdm(range(1, num_epochs + 1)):
    # Forward pass to get node embeddings in hyperbolic space
    node_emb_hyperbolic = model(features, norm_adj)  # [num_nodes, hidden_dim] in hyperbolic space

    # Training
    train_loss = train_epoch(model, optimizer, criterion, train_heads, train_tails, train_relations, node_emb_hyperbolic)

    # Evaluation on Validation Set
    val_loss, val_accuracy = evaluate(model, criterion, valid_heads, valid_tails, valid_relations, node_emb_hyperbolic)

    # Early Stopping and Best Model Tracking
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        # Evaluate on Test Set
        test_loss, test_accuracy = evaluate(model, criterion, test_heads, test_tails, test_relations, node_emb_hyperbolic)
        best_test_accuracy = test_accuracy

    # Print Epoch Results
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_accuracy:.4f}, Best Test Acc={best_test_accuracy:.4f}, Curvature: {model.gcn.manifold.c.item():.6f}")

    # Save the curvature, losses, and accuracies
    curvatures.append(model.gcn.manifold.c.item())
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    test_accuracies.append(test_accuracy)

    # Visualize embeddings every 100 epochs
    if epoch % 100 == 0:
        # Project the hyperbolic embeddings to Euclidean space using expmap0
        node_emb_euclidean = model.gcn.manifold.expmap0(node_emb_hyperbolic)  # Map hyperbolic to Euclidean space

        # Reduce the dimensionality of the embeddings to 2D for visualization
        node_emb_2d = pca.fit_transform(node_emb_euclidean.cpu().detach().numpy())

        # Create the scatter plot
        plt.figure(figsize=(8, 6))
        plt.scatter(node_emb_2d[:, 0], node_emb_2d[:, 1], alpha=0.7, edgecolors='k', s=10)
        plt.title(f"Embeddings at Epoch {epoch}")
        plt.xlabel("Dimension 1")
        plt.ylabel("Dimension 2")
        plt.xlim(-2, 2)  # Adjust as needed
        plt.ylim(-2, 2)  # Adjust as needed
        plt.show()

print("\nTraining Completed.")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")
print(f"Corresponding Test Accuracy: {best_test_accuracy:.4f}")
print(f"Curvature: {model.gcn.manifold.c.item():.6f}")


## DISEASE

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from torch import optim
import geoopt
from geoopt import ManifoldParameter
import os
from tqdm import tqdm
import pandas as pd
from scipy.sparse import csr_matrix, diags
import numpy as np
import random
from sklearn.model_selection import train_test_split
import scipy.sparse as sp

##############################################
# Data Loading and Preprocessing
##############################################

def load_synthetic_data(dataset_str, use_feats, data_path):
    object_to_idx = {}
    idx_counter = 0
    edges = []

    # Read all edges from the CSV file
    with open(os.path.join(data_path, f"{dataset_str}.edges.csv"), 'r') as f:
        all_edges = f.readlines()

    for line in all_edges:
        n1, n2 = line.rstrip().split(',')

        # Assign unique indices to nodes
        if n1 in object_to_idx:
            i = object_to_idx[n1]
        else:
            i = idx_counter
            object_to_idx[n1] = i
            idx_counter += 1

        if n2 in object_to_idx:
            j = object_to_idx[n2]
        else:
            j = idx_counter
            object_to_idx[n2] = j
            idx_counter += 1

        edges.append((i, j))

    num_nodes = len(object_to_idx)

    # Initialize adjacency matrix using LIL format for efficient assignment
    adj = sp.lil_matrix((num_nodes, num_nodes), dtype=np.float32)

    for i, j in edges:
        adj[i, j] = 1.0  # Undirected graph: set both (i, j) and (j, i)
        adj[j, i] = 1.0

    # Convert adjacency matrix to CSR format for efficient arithmetic operations
    adj = adj.tocsr()

    # Load features
    if use_feats:
        features = sp.load_npz(os.path.join(data_path, f"{dataset_str}.feats.npz")).astype(np.float32)
    else:
        features = sp.eye(adj.shape[0], dtype=np.float32)

    # Load labels
    labels = np.load(os.path.join(data_path, f"{dataset_str}.labels.npy"))

    return adj, features, labels, object_to_idx


# Define file paths
data_path = ''  # Update this to your actual data path
dataset_str = 'disease_lp'

# Load data
adj, features, labels, object_to_idx = load_synthetic_data(dataset_str, use_feats=True, data_path=data_path)

num_nodes = adj.shape[0]
num_features = features.shape[1]
num_classes = len(np.unique(labels))  # Assuming labels are integer-encoded

print(f"Number of Nodes: {num_nodes}")
print(f"Number of Features: {num_features}")
print(f"Number of Classes: {num_classes}")

# Convert adjacency matrix to a sparse matrix and add self-loops
adj = adj + diags([1e-5] * num_nodes)

# Compute degree
degrees = np.array(adj.sum(axis=1)).flatten()

# Compute D^-0.5 (inverse square root of degree matrix)
inv_sqrt_deg = np.power(degrees, -0.5)
inv_sqrt_deg[np.isinf(inv_sqrt_deg)] = 0
D_inv_sqrt = diags(inv_sqrt_deg)

# Normalize adjacency matrix: D^-0.5 * A * D^-0.5
norm_adj = D_inv_sqrt @ adj @ D_inv_sqrt

# Convert back to a PyTorch tensor
norm_adj = torch.tensor(norm_adj.toarray(), dtype=torch.float32)

# Node Features: Load from the features matrix
features = csr_matrix(features)
features = torch.tensor(features.toarray(), dtype=torch.float32)

# Move tensors to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
features, norm_adj = features.to(device), norm_adj.to(device)

print(f"Using device: {device}")

In [ ]:
from scipy.sparse import csr_matrix, diags
import numpy as np
import torch
import random
from sklearn.model_selection import train_test_split

class HyperbolicLinear(nn.Module):
    """
    Hyperbolic linear layer using the Poincaré Ball model.
    """
    def __init__(self, manifold, in_features, out_features, bias=True):
        super(HyperbolicLinear, self).__init__()
        self.manifold = manifold
        self.in_features = in_features
        self.out_features = out_features
        # ManifoldParameter for weight in Euclidean space
        self.weight = ManifoldParameter(torch.randn(out_features, in_features) * 0.01, manifold=geoopt.Euclidean())
        if bias:
            self.bias = nn.Parameter(torch.zeros(out_features))
        else:
            self.register_parameter('bias', None)

    def forward(self, x):
        # x is on manifold. Map to tangent space at 0, apply linear, map back
        x_tan = self.manifold.logmap0(x, dim=-1)
        out = F.linear(x_tan, self.weight, self.bias)
        out = self.manifold.expmap0(out, dim=-1)
        return out

class HyperbolicGCNLayer(nn.Module):
    def __init__(self, manifold, in_features, out_features):
        super(HyperbolicGCNLayer, self).__init__()
        self.manifold = manifold
        self.lin = HyperbolicLinear(manifold, in_features, out_features)

    def forward(self, x, adj):
        # x on manifold
        x_tan = self.manifold.logmap0(x, dim=-1)
        x_agg_tan = adj @ x_tan  # Aggregate in tangent space
        x_agg = self.manifold.expmap0(x_agg_tan, dim=-1)
        return self.lin(x_agg)

class HyperbolicGCN(nn.Module):
    def __init__(self, num_features, hidden_dim, c=1.0):
        super(HyperbolicGCN, self).__init__()
        # Initialize the manifold inside the model to ensure its parameters are included
        self.manifold = geoopt.PoincareBall(c=c, learnable=True)
        self.layer1 = HyperbolicGCNLayer(self.manifold, num_features, hidden_dim)
        self.layer2 = HyperbolicGCNLayer(self.manifold, hidden_dim, hidden_dim)

    def forward(self, x, adj):
        x = self.manifold.expmap0(x, dim=-1)  # Ensure x is on manifold
        x = self.layer1(x, adj)
        # Hyperbolic activation: apply tanh in tangent space
        x_tan = self.manifold.logmap0(x, dim=-1)
        x_tan = torch.tanh(x_tan)
        x = self.manifold.expmap0(x_tan, dim=-1)
        x = self.layer2(x, adj)
        return x

class LinkPredictionModel(nn.Module):
    def __init__(self, num_features, hidden_dim, num_relations, c=1.0):
        super(LinkPredictionModel, self).__init__()
        self.gcn = HyperbolicGCN(num_features, hidden_dim, c=c)
        # Define a classifier that takes concatenated head and tail embeddings
        self.classifier = nn.Linear(hidden_dim * 2, num_relations)  # Multi-class classification
        nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, x, adj):
        node_emb = self.gcn(x, adj)  # [num_nodes, hidden_dim]
        return node_emb

    def classify(self, head_idx, tail_idx, node_emb):
        # Retrieve embeddings
        head_emb = node_emb[head_idx]     # [batch_size, hidden_dim]
        tail_emb = node_emb[tail_idx]     # [batch_size, hidden_dim]
        # Concatenate head and tail embeddings
        combined = torch.cat([head_emb, tail_emb], dim=1)  # [batch_size, hidden_dim * 2]
        scores = self.classifier(combined)  # [batch_size, num_relations]
        return scores

##############################################
# Training and Evaluation Utilities
##############################################

def prepare_data(triples):
    """
    Prepare tensors for head, tail, and relation indices.
    """
    triples = torch.tensor(triples, dtype=torch.long, device=device)
    heads = triples[:, 0]
    relations = triples[:, 1]
    tails = triples[:, 2]
    return heads, tails, relations

def train_epoch(model, optimizer, criterion, heads, tails, relations, node_emb):
    model.train()
    optimizer.zero_grad()

    # Get relation scores
    scores = model.classify(heads, tails, node_emb)  # [batch_size, num_relations]

    # Compute loss
    loss = criterion(scores, relations)
    loss.backward()
    optimizer.step()

    return loss.item()

def evaluate(model, criterion, heads, tails, relations, node_emb):
    model.eval()
    with torch.no_grad():
        scores = model.classify(heads, tails, node_emb)  # [batch_size, num_relations]
        loss = criterion(scores, relations).item()
        preds = torch.argmax(scores, dim=1)
        correct = (preds == relations).sum().item()
        total = relations.size(0)
        accuracy = correct / total
    return loss, accuracy

##############################################
# Training Loop
##############################################

# Generate Train, Validation, and Test Splits for Link Prediction

def generate_negative_edges(adj_matrix, num_neg_samples, excluded_edges=set()):
    neg_edges = set()
    while len(neg_edges) < num_neg_samples:
        i = random.randint(0, num_nodes - 1)
        j = random.randint(0, num_nodes - 1)
        if i == j:
            continue
        if adj_matrix[i, j] == 0 and (i, j) not in excluded_edges and (j, i) not in excluded_edges:
            neg_edges.add((i, j))
    return list(neg_edges)

# Extract existing edges
existing_edges = set()
adj_coo = adj.tocoo()
for i, j in zip(adj_coo.row, adj_coo.col):
    if i < j:  # To avoid duplicates in undirected graph
        existing_edges.add((i, j))

# Number of positive and negative samples
num_positive = len(existing_edges)
num_negative = num_positive  # Balance the dataset

# Generate negative edges
negative_edges = generate_negative_edges(adj, num_negative, excluded_edges=existing_edges)

# Create labels: 1 for positive, 0 for negative
positive_labels = np.ones(num_positive)
negative_labels = np.zeros(num_negative)

# Combine positive and negative edges
all_edges = list(existing_edges) + negative_edges
all_labels = np.concatenate([positive_labels, negative_labels])

# Split into train, val, test
train_edges, temp_edges, train_labels, temp_labels = train_test_split(all_edges, all_labels, test_size=0.3, random_state=42, stratify=all_labels)
val_edges, test_edges, val_labels, test_labels = train_test_split(temp_edges, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels)

# Prepare training, validation, and test data
train_triples = [(h, r, t) for (h, t), r in zip(train_edges, train_labels)]
val_triples = [(h, r, t) for (h, t), r in zip(val_edges, val_labels)]
test_triples = [(h, r, t) for (h, t), r in zip(test_edges, test_labels)]

train_heads, train_tails, train_relations = prepare_data(train_triples)
val_heads, val_tails, val_relations = prepare_data(val_triples)
test_heads, test_tails, test_relations = prepare_data(test_triples)

# Initialize the model
hidden_dim = 128
num_relations = len(np.unique(train_labels))
model = LinkPredictionModel(num_features=num_features, hidden_dim=hidden_dim, num_relations=num_relations, c=1).to(device)

# Initialize input: ensure features are on the manifold
with torch.no_grad():
    x = model.gcn.manifold.expmap0(features, dim=-1)

# Optimizer Choices:
riemannian_adam = geoopt.optim.RiemannianAdam(model.parameters(), lr=1e-4)
riemannian_sgd = geoopt.optim.RiemannianSGD(model.parameters(), lr=1e-2)
riemannian_mixed_precision = MixedPrecisionRiemannianAdam(model.parameters(), lr=1e-2)
riemannian_adagrad = RiemannianAdagrad(model.parameters(), lr=1e-2) # error currently

# SET OPTIMIZER:
optimizer = riemannian_adam

# Define loss function for multi-class classification
criterion = nn.CrossEntropyLoss()

# Training parameters
num_epochs = 500
best_val_accuracy = 0.0
best_test_accuracy = 0.0

curvatures, train_losses, val_losses, val_accuracies, test_accuracies = [], [], [], [], []

print("\nStarting Training...\n")
for epoch in tqdm(range(1, num_epochs + 1)):
    # Forward pass to get node embeddings
    node_emb = model(features, norm_adj)  # [num_nodes, hidden_dim]

    # Training
    train_loss = train_epoch(model, optimizer, criterion, train_heads, train_tails, train_relations, node_emb)

    # Evaluation on Validation Set
    val_loss, val_accuracy = evaluate(model, criterion, val_heads, val_tails, val_relations, node_emb)

    # Early Stopping and Best Model Tracking
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        # Evaluate on Test Set
        test_loss, test_accuracy = evaluate(model, criterion, test_heads, test_tails, test_relations, node_emb)
        best_test_accuracy = test_accuracy

    # Print Epoch Results
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_accuracy:.4f}, Best Test Acc={best_test_accuracy:.4f}, Curvature: {model.gcn.manifold.c.item():.6f}")

    curvatures.append(model.gcn.manifold.c.item())
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    test_accuracies.append(test_accuracy)

    # Visualize embeddings every 10 epochs (or at any frequency)
    #if epoch % 1000 == 0:
        #visualize_embeddings_in_poincare(node_emb, labels, model.gcn.manifold, epoch=epoch)

print("\nTraining Completed.")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")
print(f"Corresponding Test Accuracy: {best_test_accuracy:.4f}")
print(f"Curvature: {model.gcn.manifold.c.item():.6f}")

In [ ]:
##########################################################
# PLOTTING
##########################################################
import matplotlib.pyplot as plt
epochs = list(range(1, num_epochs + 1))
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot training loss and test accuracy on the primary y-axis
ax1.plot(epochs, train_losses, label="Train Loss", color="blue", linestyle="-")
ax1.plot(epochs, test_accuracies, label="Best Test Accuracy", color="red", linestyle="--")
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Train Loss / Best Test Accuracy", color="black")
ax1.tick_params(axis="y", labelcolor="black")
ax1.legend(loc="upper left")


# Add a horizontal line at y=1
ax1.axhline(y=1, color="lightcoral", linestyle="--", linewidth=1, label="y=1")

# Add a horizontal line at y=0
ax1.axhline(y=0, color="grey", linestyle="--", linewidth=1, label="y=1")

# Plot curvature on a secondary y-axis
ax2 = ax1.twinx()
ax2.plot(epochs, curvatures, label="Curvature", color="green", linestyle=":")
ax2.set_ylabel("Curvature", color="green")
ax2.tick_params(axis="y", labelcolor="green")
ax2.legend(loc="upper right")

# Add title and grid
plt.title("Training Progress: Train Loss, Best Test Accuracy, and Curvature")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.ylim(0, 1.5)
plt.tight_layout()
plt.show()

## CORA and PUBMED

In [ ]:
# Define file paths
data_path = ''  # **Update this to your actual data path**
dataset_str = 'pubmed'  # cora or pubmed

import os
import sys
import pickle as pkl
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from torch import optim
import geoopt
from geoopt import ManifoldParameter
from tqdm import tqdm
import pandas as pd
from scipy.sparse import csr_matrix, diags, vstack
import numpy as np
import random
from sklearn.model_selection import train_test_split
import scipy.sparse as sp

##############################################
# Auxiliary Functions
##############################################

def parse_index_file(filename):
    """Parse index file to list of indices."""
    index = []
    with open(filename, 'r') as f:
        for line in f:
            index.append(int(line.strip()))
    return index

##############################################
# Data Loading and Preprocessing
##############################################

def load_citation_data(dataset_str, use_feats, data_path, split_seed=None):
    """
    Load citation network dataset (Cora, Citeseer, Pubmed).

    Parameters:
    - dataset_str: Name of the dataset (e.g., 'cora')
    - use_feats: Whether to use features or identity matrix
    - data_path: Path to the dataset files
    - split_seed: Seed for splitting data (optional)

    Returns:
    - adj: Adjacency matrix (scipy sparse CSR matrix)
    - features: Feature matrix (scipy sparse CSR matrix)
    - labels: Numpy array of labels
    - idx_train: List of training indices
    - idx_val: List of validation indices
    - idx_test: List of test indices
    """
    names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
    objects = []
    for name in names:
        file_path = os.path.join(data_path, f"ind.{dataset_str}.{name}")
        with open(file_path, 'rb') as f:
            if sys.version_info > (3, 0):
                out = pkl.load(f, encoding='latin1')
            else:
                out = pkl.load(f)
            objects.append(out)

    x, y, tx, ty, allx, ally, graph = tuple(objects)

    test_idx_reorder = parse_index_file(os.path.join(data_path, f"ind.{dataset_str}.test.index"))
    test_idx_range = np.sort(test_idx_reorder)

    # Handle feature matrices
    if isinstance(allx, sp.csr_matrix):
        features = sp.vstack((allx, tx)).tolil()
    else:
        features = sp.vstack((allx, tx)).tolil()

    features[test_idx_reorder, :] = features[test_idx_range, :]

    # Handle labels
    labels = np.vstack((ally, ty))
    labels[test_idx_reorder, :] = labels[test_idx_range, :]
    labels = np.argmax(labels, axis=1)

    # Define train, validation, and test indices
    idx_test = test_idx_range.tolist()
    idx_train = list(range(len(y)))
    idx_val = list(range(len(y), len(y) + 500))  # Adjust the validation size as needed

    # Create adjacency matrix
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))
    if not use_feats:
        features = sp.eye(adj.shape[0])

    return adj, features, labels, idx_train, idx_val, idx_test

# Load data using load_citation_data
adj, features, labels, idx_train, idx_val, idx_test = load_citation_data(
    dataset_str=dataset_str,
    use_feats=True,
    data_path=data_path
)

num_nodes = adj.shape[0]
num_features = features.shape[1]
num_classes = len(np.unique(labels))  # Assuming labels are integer-encoded

print(f"Number of Nodes: {num_nodes}")
print(f"Number of Features: {num_features}")
print(f"Number of Classes: {num_classes}")

# Convert adjacency matrix to a sparse matrix and add self-loops
adj = adj + diags([1e-5] * num_nodes)  # Small value to prevent singularity

# Compute degree
degrees = np.array(adj.sum(axis=1)).flatten()

# Compute D^-0.5 (inverse square root of degree matrix)
inv_sqrt_deg = np.power(degrees, -0.5)
inv_sqrt_deg[np.isinf(inv_sqrt_deg)] = 0
D_inv_sqrt = diags(inv_sqrt_deg)

# Normalize adjacency matrix: D^-0.5 * A * D^-0.5
norm_adj = D_inv_sqrt.dot(adj).dot(D_inv_sqrt)

# Convert back to a PyTorch tensor
norm_adj = torch.tensor(norm_adj.toarray(), dtype=torch.float32)

# Node Features: Load from the features matrix
features = features.tocsr()
features = torch.tensor(features.toarray(), dtype=torch.float32)

# Move tensors to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
features, norm_adj = features.to(device), norm_adj.to(device)

print(f"Using device: {device}")

# Generate Train, Validation, and Test Splits for Link Prediction

def generate_negative_edges(adj_matrix, num_neg_samples, excluded_edges=set()):
    """
    Generate negative edges (non-existing edges) for link prediction.

    Parameters:
    - adj_matrix: scipy sparse matrix (CSR format)
    - num_neg_samples: Number of negative samples to generate
    - excluded_edges: Set of edges to exclude (existing edges)

    Returns:
    - List of tuples representing negative edges
    """
    neg_edges = set()
    while len(neg_edges) < num_neg_samples:
        i = random.randint(0, num_nodes - 1)
        j = random.randint(0, num_nodes - 1)
        if i == j:
            continue
        if adj_matrix[i, j] == 0 and (i, j) not in excluded_edges and (j, i) not in excluded_edges:
            neg_edges.add((i, j))
    return list(neg_edges)

# Extract existing edges
existing_edges = set()
adj_coo = adj.tocoo()
for i, j in zip(adj_coo.row, adj_coo.col):
    if i < j:  # To avoid duplicates in undirected graph
        existing_edges.add((i, j))

# Number of positive and negative samples
num_positive = len(existing_edges)
num_negative = num_positive  # Balance the dataset

# Generate negative edges
negative_edges = generate_negative_edges(adj, num_negative, excluded_edges=existing_edges)

# Create labels: 1 for positive, 0 for negative
positive_labels = np.ones(num_positive)
negative_labels = np.zeros(num_negative)

# Combine positive and negative edges
all_edges = list(existing_edges) + negative_edges
all_labels = np.concatenate([positive_labels, negative_labels])

# Split into train, val, test
train_edges, temp_edges, train_labels, temp_labels = train_test_split(
    all_edges, all_labels, test_size=0.3, random_state=42, stratify=all_labels
)
val_edges, test_edges, val_labels, test_labels = train_test_split(
    temp_edges, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

print(f"Train edges: {len(train_edges)}, Val edges: {len(val_edges)}, Test edges: {len(test_edges)}")


In [ ]:
##############################################
# Training Loop
##############################################

def prepare_edge_data(edges, labels):
    """
    Prepare tensors for head, tail, and labels.
    """
    heads = torch.tensor([edge[0] for edge in edges], dtype=torch.long, device=device)
    tails = torch.tensor([edge[1] for edge in edges], dtype=torch.long, device=device)
    labels = torch.tensor(labels, dtype=torch.long, device=device)  # Use torch.long for CrossEntropyLoss
    return heads, tails, labels

# Prepare training, validation, and test data
train_heads, train_tails, train_labels = prepare_edge_data(train_edges, train_labels)
val_heads, val_tails, val_labels = prepare_edge_data(val_edges, val_labels)
test_heads, test_tails, test_labels = prepare_edge_data(test_edges, test_labels)

# Initialize the model
hidden_dim = 128  # Reduced hidden dimension
num_relations = len(np.unique(train_labels.cpu().numpy()))
model = LinkPredictionModel(num_features=num_features, hidden_dim=hidden_dim, num_relations = num_relations, c=1).to(device)

# Initialize input: ensure features are on the manifold
with torch.no_grad():
    x = model.gcn.manifold.expmap0(features, dim=-1)

# Optimizer Choices:
riemannian_adam = geoopt.optim.RiemannianAdam(model.parameters(), lr=1e-3)
riemannian_sgd = geoopt.optim.RiemannianSGD(model.parameters(), lr=5e-2)
riemannian_mixed_precision = MixedPrecisionRiemannianAdam(model.parameters(), lr=1e-2)
riemannian_adagrad = RiemannianAdagrad(model.parameters(), lr=1e-2) # error currently

# SET OPTIMIZER:
optimizer = riemannian_mixed_precision

# Define loss function for binary classification
criterion = nn.CrossEntropyLoss()

# Training parameters
num_epochs = 500
best_val_accuracy = 0.0
best_test_accuracy = 0.0

curvatures, train_losses, val_losses, val_accuracies, test_accuracies = [], [], [], [], []

print("\nStarting Training...\n")
for epoch in tqdm(range(1, num_epochs + 1)):
    # Forward pass to get node embeddings
    node_emb = model(features, norm_adj)  # [num_nodes, hidden_dim]

    # Training
    train_loss = train_epoch(model, optimizer, criterion, train_heads, train_tails, train_labels, node_emb)

    # Evaluation on Validation Set
    val_loss, val_accuracy = evaluate(model, criterion, val_heads, val_tails, val_labels, node_emb)

    # Early Stopping and Best Model Tracking
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        # Evaluate on Test Set
        test_loss, test_accuracy = evaluate(model, criterion, test_heads, test_tails, test_labels, node_emb)
        best_test_accuracy = test_accuracy

    # Print Epoch Results
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_accuracy:.4f}, Best Test Acc={best_test_accuracy:.4f}, Curvature: {model.gcn.manifold.c.item():.6f}")

    curvatures.append(model.gcn.manifold.c.item())
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    test_accuracies.append(test_accuracy)

print("\nTraining Completed.")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")
print(f"Corresponding Test Accuracy: {best_test_accuracy:.4f}")
print(f"Curvature: {model.gcn.manifold.c.item():.6f}")


In [ ]:
##########################################################
# PLOTTING
##########################################################
import matplotlib.pyplot as plt
epochs = list(range(1, num_epochs + 1))
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot training loss and test accuracy on the primary y-axis
ax1.plot(epochs, train_losses, label="Train Loss", color="blue", linestyle="-")
ax1.plot(epochs, test_accuracies, label="Best Test Accuracy", color="red", linestyle="--")
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Train Loss / Best Test Accuracy", color="black")
ax1.tick_params(axis="y", labelcolor="black")
ax1.legend(loc="upper left")


# Add a horizontal line at y=1
ax1.axhline(y=1, color="lightcoral", linestyle="--", linewidth=1, label="y=1")

# Add a horizontal line at y=0
ax1.axhline(y=0, color="grey", linestyle="--", linewidth=1, label="y=1")

# Plot curvature on a secondary y-axis
ax2 = ax1.twinx()
ax2.plot(epochs, curvatures, label="Curvature", color="green", linestyle=":")
ax2.set_ylabel("Curvature", color="green")
ax2.tick_params(axis="y", labelcolor="green")
ax2.legend(loc="upper right")

# Add title and grid
plt.title("Training Progress: Train Loss, Best Test Accuracy, and Curvature")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.ylim(0, 1.5)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch

def visualize_embeddings_in_poincare(node_emb, labels, manifold, epoch=None):
    """
    Visualizes the node embeddings in the Poincaré disk using t-SNE for dimensionality reduction.
    It maps the hyperbolic embeddings to Euclidean space, reduces them to 2D, and visualizes them.

    Parameters:
    - node_emb: Tensor of node embeddings from the model [num_nodes, hidden_dim]
    - labels: Array of node labels [num_nodes]
    - manifold: The manifold (e.g., PoincaréBall) used for the embeddings
    - epoch: The current epoch (optional, used in title)
    """
    # Map to Euclidean space using logmap
    node_emb_euclidean = manifold.logmap0(node_emb, dim=-1)

    # Reduce to 2D using t-SNE
    tsne = TSNE(n_components=2, random_state=42)
    node_emb_2d = tsne.fit_transform(node_emb_euclidean.detach().cpu().numpy())

    # Normalize to fit within the unit disk for visualization
    norm = np.linalg.norm(node_emb_2d, axis=1, keepdims=True)
    node_emb_2d = node_emb_2d / norm  # Keep points within the unit disk

    # Plot the embeddings
    plt.figure(figsize=(8, 8))
    plt.scatter(node_emb_2d[:, 0], node_emb_2d[:, 1], c=labels, cmap="viridis", s=30, alpha=0.7)
    plt.colorbar()  # Show color bar to indicate label classes

    # Draw the boundary of the Poincaré disk (unit circle)
    circle = plt.Circle((0, 0), 1, color='r', fill=False, linestyle='--', linewidth=2)
    plt.gca().add_artist(circle)

    # Set plot limits and aspect
    plt.title(f"Node Embeddings in the Poincaré Disk (Epoch {epoch})" if epoch else "Node Embeddings in the Poincaré Disk")
    plt.axis('equal')
    plt.xlim(-1, 1)
    plt.ylim(-1, 1)
    plt.show()

def hyperbolic_to_euclidean(embedding):
    """Project hyperbolic embeddings from the Poincaré disk to Euclidean space."""
    x, y = embedding[:, 0], embedding[:, 1]
    norm = x**2 + y**2
    x_euclidean = 2 * x / (1 + norm)
    y_euclidean = 2 * y / (1 + norm)
    return np.stack([x_euclidean, y_euclidean], axis=1)


import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch

def visualize_embeddings_in_batches_no_normalization(node_emb, labels, manifold, batch_size=1024, epoch=None):
    """
    Visualizes the node embeddings in 2D using t-SNE, processing the data in batches.
    It maps the hyperbolic embeddings to Euclidean space, reduces them to 2D, and visualizes them.

    Parameters:
    - node_emb: Tensor of node embeddings from the model [num_nodes, hidden_dim]
    - labels: Array of node labels [num_nodes]
    - manifold: The manifold (e.g., PoincaréBall) used for the embeddings
    - batch_size: The batch size for processing (default is 1024)
    - epoch: The current epoch (optional, used in title)
    """
    # Create lists to store batch results
    batch_node_emb_2d = []
    batch_labels = []

    # Process the embeddings in batches
    num_batches = (len(node_emb) + batch_size - 1) // batch_size  # Calculate the number of batches
    for i in range(num_batches):
        # Select the batch (slice the embeddings and labels)
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(node_emb))

        batch_emb = node_emb[start_idx:end_idx]
        batch_lbl = labels[start_idx:end_idx]

        # Map the embeddings to Euclidean space using logmap
        batch_emb_euclidean = manifold.logmap0(batch_emb, dim=-1)

        # Reduce the embeddings to 2D using t-SNE
        tsne = TSNE(n_components=2, random_state=42)
        batch_emb_2d = tsne.fit_transform(batch_emb_euclidean.detach().cpu().numpy())

        # Skip normalization for now (visualize in original space)
        batch_node_emb_2d.append(batch_emb_2d)
        batch_labels.append(batch_lbl)

    # Concatenate the results from all batches
    node_emb_2d = np.concatenate(batch_node_emb_2d, axis=0)
    labels = np.concatenate(batch_labels, axis=0)

    # Plot the embeddings
    plt.figure(figsize=(8, 8))
    plt.scatter(node_emb_2d[:, 0], node_emb_2d[:, 1], c=labels, cmap="viridis", s=30, alpha=0.7)
    plt.colorbar()  # Show color bar to indicate label classes

    # Set plot limits and aspect
    plt.title(f"Node Embeddings (Epoch {epoch})" if epoch else "Node Embeddings")
    plt.axis('equal')
    plt.show()


## BASELINE COMPARISON

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from torch import optim
import geoopt
from geoopt import ManifoldParameter
import os
from tqdm import tqdm
import pandas as pd
from scipy.sparse import csr_matrix, diags
import numpy as np
import random
from sklearn.model_selection import train_test_split
import scipy.sparse as sp

##############################################
# Data Loading and Preprocessing
##############################################

def load_synthetic_data(dataset_str, use_feats, data_path):
    object_to_idx = {}
    idx_counter = 0
    edges = []

    # Read all edges from the CSV file
    with open(os.path.join(data_path, f"{dataset_str}.edges.csv"), 'r') as f:
        all_edges = f.readlines()

    for line in all_edges:
        n1, n2 = line.rstrip().split(',')

        # Assign unique indices to nodes
        if n1 in object_to_idx:
            i = object_to_idx[n1]
        else:
            i = idx_counter
            object_to_idx[n1] = i
            idx_counter += 1

        if n2 in object_to_idx:
            j = object_to_idx[n2]
        else:
            j = idx_counter
            object_to_idx[n2] = j
            idx_counter += 1

        edges.append((i, j))

    num_nodes = len(object_to_idx)

    # Initialize adjacency matrix using LIL format for efficient assignment
    adj = sp.lil_matrix((num_nodes, num_nodes), dtype=np.float32)

    for i, j in edges:
        adj[i, j] = 1.0  # Undirected graph: set both (i, j) and (j, i)
        adj[j, i] = 1.0

    # Convert adjacency matrix to CSR format for efficient arithmetic operations
    adj = adj.tocsr()

    # Load features
    if use_feats:
        features = sp.load_npz(os.path.join(data_path, f"{dataset_str}.feats.npz")).astype(np.float32)
    else:
        features = sp.eye(adj.shape[0], dtype=np.float32)

    # Load labels
    labels = np.load(os.path.join(data_path, f"{dataset_str}.labels.npy"))

    return adj, features, labels, object_to_idx


# Define file paths
data_path = ''  # Update this to your actual data path
dataset_str = 'disease_lp'

# Load data
adj, features, labels, object_to_idx = load_synthetic_data(dataset_str, use_feats=True, data_path=data_path)

num_nodes = adj.shape[0]
num_features = features.shape[1]
num_classes = len(np.unique(labels))  # Assuming labels are integer-encoded

print(f"Number of Nodes: {num_nodes}")
print(f"Number of Features: {num_features}")
print(f"Number of Classes: {num_classes}")

# Convert adjacency matrix to a sparse matrix and add self-loops
adj = adj + diags([1e-5] * num_nodes)

# Compute degree
degrees = np.array(adj.sum(axis=1)).flatten()

# Compute D^-0.5 (inverse square root of degree matrix)
inv_sqrt_deg = np.power(degrees, -0.5)
inv_sqrt_deg[np.isinf(inv_sqrt_deg)] = 0
D_inv_sqrt = diags(inv_sqrt_deg)

# Normalize adjacency matrix: D^-0.5 * A * D^-0.5
norm_adj = D_inv_sqrt @ adj @ D_inv_sqrt

# Convert back to a PyTorch tensor
norm_adj = torch.tensor(norm_adj.toarray(), dtype=torch.float32)

# Node Features: Load from the features matrix
features = csr_matrix(features)
features = torch.tensor(features.toarray(), dtype=torch.float32)

# Move tensors to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
features, norm_adj = features.to(device), norm_adj.to(device)

print(f"Using device: {device}")


###################################################################################
# GCN DEFINITION
###################################################################################

# Define the GCN Baseline Model
class GCNBaseline(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_relations):
        super(GCNBaseline, self).__init__()
        self.conv1 = nn.Linear(input_dim, hidden_dim)
        self.conv2 = nn.Linear(hidden_dim, hidden_dim)
        self.classifier = nn.Linear(hidden_dim * 2, num_relations)  # For link prediction

    def forward(self, features, adj):
        x = F.relu(self.conv1(features))
        x = torch.matmul(adj, x)  # Graph convolution using adjacency matrix
        x = F.relu(self.conv2(x))
        return x

    def classify(self, head_idx, tail_idx, node_emb):
        head_emb = node_emb[head_idx]
        tail_emb = node_emb[tail_idx]
        score = self.classifier(torch.cat([head_emb, tail_emb], dim=1))
        return score

# Training function
def train_epoch(model, optimizer, criterion, heads, tails, relations, node_emb):
    model.train()
    optimizer.zero_grad()

    # Get predictions for head-tail pairs
    scores = model.classify(heads, tails, node_emb)  # [batch_size, num_relations]

    # Compute loss
    loss = criterion(scores, relations)

    # Backpropagation
    loss.backward()
    optimizer.step()

    return loss.item()

# Link Prediction Model for Hyperbolic Graph Convolution
class LinkPredictionModel(nn.Module):
    def __init__(self, num_nodes, embedding_dim, num_relations, manifold, c):
        super(LinkPredictionModel, self).__init__()
        self.gcn = GCNBaseline(embedding_dim, embedding_dim, num_relations)
        self.relation_embeddings = nn.Embedding(num_relations, embedding_dim)
        self.manifold = manifold

    def forward(self, features, adj):
        # Generate node embeddings
        node_embeddings = self.gcn(features, adj)
        return node_embeddings

    def classify(self, head_idx, tail_idx, node_embeddings):
        # Calculate scores for all relations
        head_emb = node_embeddings[head_idx]
        tail_emb = node_embeddings[tail_idx]
        scores = torch.matmul(head_emb, self.relation_embeddings.weight.T)
        scores += torch.matmul(tail_emb, self.relation_embeddings.weight.T)
        return scores

############################################################
#  TRAINING FUNCTIONS
############################################################

def prepare_data(triples):
    """
    Prepare tensors for head, tail, and relation indices.
    """
    triples = torch.tensor(triples, dtype=torch.long, device=device)
    heads = triples[:, 0]
    relations = triples[:, 1]
    tails = triples[:, 2]
    return heads, tails, relations

def train_epoch(model, optimizer, criterion, heads, tails, relations, node_emb):
    model.train()
    optimizer.zero_grad()

    # Get relation scores
    scores = model.classify(heads, tails, node_emb)  # [batch_size, num_relations]

    # Compute loss
    loss = criterion(scores, relations)
    loss.backward()
    optimizer.step()

    return loss.item()

def evaluate(model, criterion, heads, tails, relations, node_emb):
    model.eval()
    with torch.no_grad():
        scores = model.classify(heads, tails, node_emb)  # [batch_size, num_relations]
        loss = criterion(scores, relations).item()
        preds = torch.argmax(scores, dim=1)
        correct = (preds == relations).sum().item()
        total = relations.size(0)
        accuracy = correct / total
    return loss, accuracy

def generate_negative_edges(adj_matrix, num_neg_samples, excluded_edges=set()):
    neg_edges = set()
    while len(neg_edges) < num_neg_samples:
        i = random.randint(0, num_nodes - 1)
        j = random.randint(0, num_nodes - 1)
        if i == j:
            continue
        if adj_matrix[i, j] == 0 and (i, j) not in excluded_edges and (j, i) not in excluded_edges:
            neg_edges.add((i, j))
    return list(neg_edges)

# Extract existing edges
existing_edges = set()
adj_coo = adj.tocoo()
for i, j in zip(adj_coo.row, adj_coo.col):
    if i < j:  # To avoid duplicates in undirected graph
        existing_edges.add((i, j))

# Number of positive and negative samples
num_positive = len(existing_edges)
num_negative = num_positive  # Balance the dataset

# Generate negative edges
negative_edges = generate_negative_edges(adj, num_negative, excluded_edges=existing_edges)

# Create labels: 1 for positive, 0 for negative
positive_labels = np.ones(num_positive)
negative_labels = np.zeros(num_negative)

# Combine positive and negative edges
all_edges = list(existing_edges) + negative_edges
all_labels = np.concatenate([positive_labels, negative_labels])

# Split into train, val, test
train_edges, temp_edges, train_labels, temp_labels = train_test_split(all_edges, all_labels, test_size=0.3, random_state=42, stratify=all_labels)
val_edges, test_edges, val_labels, test_labels = train_test_split(temp_edges, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels)

# Prepare training, validation, and test data
train_triples = [(h, r, t) for (h, t), r in zip(train_edges, train_labels)]
val_triples = [(h, r, t) for (h, t), r in zip(val_edges, val_labels)]
test_triples = [(h, r, t) for (h, t), r in zip(test_edges, test_labels)]

train_heads, train_tails, train_relations = prepare_data(train_triples)
val_heads, val_tails, val_relations = prepare_data(val_triples)
test_heads, test_tails, test_relations = prepare_data(test_triples)

# Initialize the model
hidden_dim = 128
num_relations = len(np.unique(train_labels))
model = LinkPredictionModel(num_features=num_features, hidden_dim=hidden_dim, num_relations=num_relations, c=1).to(device)

# Initialize input: ensure features are on the manifold
with torch.no_grad():
    x = model.gcn.manifold.expmap0(features, dim=-1)

# Optimizer Choices:
riemannian_adam = geoopt.optim.RiemannianAdam(model.parameters(), lr=1e-4)
riemannian_sgd = geoopt.optim.RiemannianSGD(model.parameters(), lr=1e-2)
riemannian_mixed_precision = MixedPrecisionRiemannianAdam(model.parameters(), lr=1e-2)
riemannian_adagrad = RiemannianAdagrad(model.parameters(), lr=1e-2) # error currently

# SET OPTIMIZER:
optimizer = riemannian_adam

# Define loss function for multi-class classification
criterion = nn.CrossEntropyLoss()

# Training parameters
num_epochs = 500
best_val_accuracy = 0.0
best_test_accuracy = 0.0

curvatures, train_losses, val_losses, val_accuracies, test_accuracies = [], [], [], [], []

print("\nStarting Training...\n")
for epoch in tqdm(range(1, num_epochs + 1)):
    # Forward pass to get node embeddings
    node_emb = model(features, norm_adj)  # [num_nodes, hidden_dim]

    # Training
    train_loss = train_epoch(model, optimizer, criterion, train_heads, train_tails, train_relations, node_emb)

    # Evaluation on Validation Set
    val_loss, val_accuracy = evaluate(model, criterion, val_heads, val_tails, val_relations, node_emb)

    # Early Stopping and Best Model Tracking
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        # Evaluate on Test Set
        test_loss, test_accuracy = evaluate(model, criterion, test_heads, test_tails, test_relations, node_emb)
        best_test_accuracy = test_accuracy

    # Print Epoch Results
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_accuracy:.4f}, Best Test Acc={best_test_accuracy:.4f}, Curvature: {model.gcn.manifold.c.item():.6f}")

    curvatures.append(model.gcn.manifold.c.item())
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    test_accuracies.append(test_accuracy)

    # Visualize embeddings every 10 epochs (or at any frequency)
    #if epoch % 1000 == 0:
        #visualize_embeddings_in_poincare(node_emb, labels, model.gcn.manifold, epoch=epoch)

print("\nTraining Completed.")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")
print(f"Corresponding Test Accuracy: {best_test_accuracy:.4f}")
print(f"Curvature: {model.gcn.manifold.c.item():.6f}")
